In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
# default_exp update

# Update

> This module contains the update algorithm for a DGLM. This follows the standard variational Bayes / Linear Bayes (VBLB) algorithm as described in *Bayesian Forecasting and Dynamic Models*, by West and Harrison (1997). Updates for a normal DLM follow a standard Kalman filter, with dynamic observation variance.

In [ ]:
#exporti
import numpy as np

In [ ]:
#exporti
def update_F(mod, X, F=None):
    if F is None:
        if mod.nregn > 0:
            mod.F[mod.iregn] = X.reshape(mod.nregn, 1)
    else:
        if mod.nregn > 0:
            # F = mod.F.copy()
            F[mod.iregn] = X.reshape(mod.nregn, 1)
        return F

## Update for a DGLM

In [ ]:
#export
def update(mod, y = None, X = None):

    # If data is missing then skip discounting and updating, posterior = prior
    if y is None or np.isnan(y):
        mod.t += 1
        mod.m = mod.a
        mod.C = mod.R

        # Get priors a, R for time t + 1 from the posteriors m, C
        mod.a = mod.G @ mod.m
        mod.R = mod.G @ mod.C @ mod.G.T
        mod.R = (mod.R + mod.R.T)/2

        mod.W = mod.get_W(X=X)

    else:

        update_F(mod, X)

        # Mean and variance
        ft, qt = mod.get_mean_and_var(mod.F, mod.a, mod.R)

        # Choose conjugate prior, match mean and variance
        mod.param1, mod.param2 = mod.get_conjugate_params(ft, qt, mod.param1, mod.param2)

        # See time t observation y (which was passed into the update function)
        mod.t += 1

        # Update the conjugate parameters and get the implied ft* and qt*
        mod.param1, mod.param2, ft_star, qt_star = mod.update_conjugate_params(y, mod.param1, mod.param2)

        # Kalman filter update on the state vector (using Linear Bayes approximation)
        mod.m = mod.a + mod.R @ mod.F * (ft_star - ft)/qt
        mod.C = mod.R - mod.R @ mod.F @ mod.F.T @ mod.R * (1 - qt_star/qt)/qt

        # Get priors a, R for time t + 1 from the posteriors m, C
        mod.a = mod.G @ mod.m
        mod.R = mod.G @ mod.C @ mod.G.T
        mod.R = (mod.R + mod.R.T)/2

        # Discount information in the time t + 1 prior
        mod.W = mod.get_W(X=X)
        mod.R = mod.R + mod.W

Testing:

In [ ]:
import numpy as np
from pybats_nbdev.dglm import dlm, pois_dglm, bern_dglm, bin_dglm
from pybats_nbdev.analysis import analysis
a0 = np.array([1, 1, 1])
R0 = np.eye(3)
mod_p = pois_dglm(a0, R0, ntrend=1, nregn=2, deltrend=1, delregn=.9)
mod_b = bin_dglm(a0, R0, ntrend=1, nregn=2, deltrend=1, delregn=.9)
mod_bern = bern_dglm(a0, R0, ntrend=1, nregn=2, deltrend=1, delregn=.9)

# New data:
y = 5
X = np.array([1,2])

# Test the Poisson DGLM
mod_p.update(y=y, X=X)
ans = np.array([[0.59974735],
   [0.59974735],
   [0.1994947 ]])
assert (np.equal(np.round(ans, 5), np.round(mod_p.a, 5)).all())

ans = np.array([-0.16107008, 0.93214436])
assert (np.equal(np.round(ans, 5), np.round(mod_p.R[0:2, 1], 5)).all())

# Test the Binomial DGLM
mod_b.update(y=y, X=X, n=10)
ans = np.array([[ 0.46543905],
   [ 0.46543905],
   [-0.0691219 ]])
assert (np.equal(np.round(ans, 5), np.round(mod_b.a, 5)).all())

ans = np.array([-0.15854342, 0.93495175])
assert (np.equal(np.round(ans, 5), np.round(mod_b.R[0:2, 1], 5)).all())

# Test the Bernoulli DGLM
mod_bern.update(y=1, X=X)
ans = np.array([[1.02626224],
                [1.02626224],
                [1.05252447]])
assert (np.equal(np.round(ans, 5), np.round(mod_bern.a, 5)).all())

ans = np.array([-1.00331466e-04,  1.11099963])
assert (np.equal(np.round(ans, 5), np.round(mod_bern.R[0:2, 1], 5)).all())


## Update for a DLM

In [ ]:
#export
def update_dlm(mod, y = None, X = None):

    # If data is missing then skip discounting and updating, posterior = prior
    if y is None or np.isnan(y):
        mod.t += 1
        mod.m = mod.a
        mod.C = mod.R

        # Get priors a, R for time t + 1 from the posteriors m, C
        mod.a = mod.G @ mod.m
        mod.R = mod.G @ mod.C @ mod.G.T
        mod.R = (mod.R + mod.R.T)/2

        mod.W = mod.get_W(X=X)

    else:
        update_F(mod, X)

        # Mean and variance
        ft, qt = mod.get_mean_and_var(mod.F, mod.a, mod.R)
        mod.param1 = ft
        mod.param2 = qt

        # See time t observation y (which was passed into the update function)
        mod.t += 1

        # Update the  parameters:
        et = y - ft

        # Adaptive coefficient vector
        At = mod.R @ mod.F / qt

        # Volatility estimate ratio
        rt = (mod.n + et**2/qt)/(mod.n + 1)

        # Kalman filter update
        mod.n = mod.n + 1
        mod.s = mod.s * rt
        mod.m = mod.a + At * et
        mod.C = rt * (mod.R - qt * At @ At.T)

        # Get priors a, R for time t + 1 from the posteriors m, C
        mod.a = mod.G @ mod.m
        mod.R = mod.G @ mod.C @ mod.G.T
        mod.R = (mod.R + mod.R.T)/2

        # Discount information
        mod.W = mod.get_W(X=X)
        mod.R = mod.R + mod.W
        mod.n = mod.delVar * mod.n

In [ ]:
from pybats_nbdev.dglm import dlm, pois_dglm, bern_dglm, bin_dglm
from pybats_nbdev.analysis import analysis
a0 = np.array([1, 1, 1])
R0 = np.eye(3)
mod_n = dlm(a0, R0, ntrend=1, nregn=2, deltrend=1, delregn=.9)

# New data:
y = 5
X = np.array([1,2])

# Test the normal DLM
mod_n.update(y = y, X=X)
ans = np.array([[1.14285714],
   [1.14285714],
   [1.28571429]])
assert(np.equal(np.round(ans, 5), np.round(mod_n.a, 5)).all())

ans = np.array([-0.08163265, 0.54421769])
assert(np.equal(np.round(ans, 5), np.round(mod_n.R[0:2,1], 5)).all())

## Update for a Binomial DGLM

In [ ]:
#export
def update_bindglm(mod, n=None, y=None, X=None):

    # If data is missing then skip discounting and updating, posterior = prior
    if y is None or np.isnan(y) or n is None or np.isnan(n) or n == 0:
        mod.t += 1
        mod.m = mod.a
        mod.C = mod.R

        # Get priors a, R for time t + 1 from the posteriors m, C
        mod.a = mod.G @ mod.m
        mod.R = mod.G @ mod.C @ mod.G.T
        mod.R = (mod.R + mod.R.T) / 2

        mod.W = mod.get_W(X=X)

    else:

        update_F(mod, X)

        # Mean and variance
        ft, qt = mod.get_mean_and_var(mod.F, mod.a, mod.R)

        # Choose conjugate prior, match mean and variance
        mod.param1, mod.param2 = mod.get_conjugate_params(ft, qt, mod.param1, mod.param2)

        # See time t observation y (which was passed into the update function)
        mod.t += 1

        # Update the conjugate parameters and get the implied ft* and qt*
        mod.param1, mod.param2, ft_star, qt_star = mod.update_conjugate_params(n, y, mod.param1, mod.param2)

        # Kalman filter update on the state vector (using Linear Bayes approximation)
        mod.m = mod.a + mod.R @ mod.F * (ft_star - ft) / qt
        mod.C = mod.R - mod.R @ mod.F @ mod.F.T @ mod.R * (1 - qt_star / qt) / qt

        # Get priors a, R for time t + 1 from the posteriors m, C
        mod.a = mod.G @ mod.m
        mod.R = mod.G @ mod.C @ mod.G.T
        mod.R = (mod.R + mod.R.T) / 2

        # Discount information in the time t + 1 prior
        mod.W = mod.get_W(X=X)
        mod.R = mod.R + mod.W

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_dglm.ipynb.
Converted 01_update.ipynb.
Converted 02_forecast.ipynb.
Converted 03_define_models.ipynb.
Converted 04_seasonal.ipynb.
Converted 05_analysis.ipynb.
Converted 06_conjugates.ipynb.
Converted 07_point_forecast.ipynb.
Converted 08_loss_functions.ipynb.
Converted 09_plot.ipynb.
Converted 10_shared.ipynb.
Converted 11_dcmm.ipynb.
Converted 12_dbcm.ipynb.
Converted 13_latent_factor.ipynb.
Converted 14_latent_factor_fxns.ipynb.
Converted index.ipynb.
